In [4]:
# 기본라이브러리 불러오기
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd
import numpy as np
import seaborn as sns

In [9]:
df = sns.load_dataset('titanic')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [11]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [12]:
df.drop(['embarked','deck'], axis=1, inplace=True)

In [15]:
df.dropna(subset=['age'], axis=0, how='any', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     714 non-null    int64   
 1   pclass       714 non-null    int64   
 2   sex          714 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        714 non-null    int64   
 5   parch        714 non-null    int64   
 6   fare         714 non-null    float64 
 7   class        714 non-null    category
 8   who          714 non-null    object  
 9   adult_male   714 non-null    bool    
 10  embark_town  712 non-null    object  
 11  alive        714 non-null    object  
 12  alone        714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [17]:
most_freq = df['embark_town'].value_counts(dropna=True).idxmax()
most_freq

'Southampton'

In [18]:
df['embark_town'].fillna(most_freq, inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     714 non-null    int64   
 1   pclass       714 non-null    int64   
 2   sex          714 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        714 non-null    int64   
 5   parch        714 non-null    int64   
 6   fare         714 non-null    float64 
 7   class        714 non-null    category
 8   who          714 non-null    object  
 9   adult_male   714 non-null    bool    
 10  embark_town  714 non-null    object  
 11  alive        714 non-null    object  
 12  alone        714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [20]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,Third,man,True,Southampton,no,True


In [24]:
# 분석에 활용할 열(속성)을 선택
ndf = df[['survived','pclass','sex','age','sibsp','parch','embark_town']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embark_town
0,0,3,male,22.0,1,0,Southampton
1,1,1,female,38.0,1,0,Cherbourg
2,1,3,female,26.0,0,0,Southampton
3,1,1,female,35.0,1,0,Southampton
4,0,3,male,35.0,0,0,Southampton


In [25]:
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis = 1)

onehot_embark = pd.get_dummies(ndf['embark_town'])
ndf = pd.concat([ndf, onehot_embark], axis = 1)

ndf.drop(['sex','embark_town'], axis =1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,Cherbourg,Queenstown,Southampton
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [26]:
# 속성 변수 선택
X = ndf[['pclass','age','sibsp','parch','female','male','Cherbourg','Queenstown','Southampton']]
y = ndf['survived']

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.3,
                                                    random_state = 10)
print(X_train.shape)
print(X_test.shape)

(499, 9)
(215, 9)


In [54]:
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=6)

# train data 가지고 모형 학습
tree_model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=6)

In [55]:
y_hat = tree_model.predict(X_test)

df = pd.DataFrame({'y_hat':y_hat,'y':y_test})
df['차이'] = df['y_hat'] == df['y']
df.head()

,y_hat,y,차이
728,0,0,True
555,0,0,True
426,1,1,True
278,0,0,True
617,0,0,True


In [56]:
len(df) - sum(df['차이'])

40

In [57]:
# 모형 성능 평가 - Confusion Matrix 계산
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)

[[119   6]
 [ 34  56]]


In [58]:
# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

              precision    recall  f1-score   support

           0       0.78      0.95      0.86       125
           1       0.90      0.62      0.74        90

    accuracy                           0.81       215
   macro avg       0.84      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215



## 모범답안

In [59]:
# 기본라이브러리 불러오기
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd
import numpy as np
import seaborn as sns

In [60]:
df = sns.load_dataset('titanic')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [61]:
df.drop(['embarked','deck'], axis=1, inplace=True)

In [62]:
df.dropna(subset=['age'], axis=0, how='any', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     714 non-null    int64   
 1   pclass       714 non-null    int64   
 2   sex          714 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        714 non-null    int64   
 5   parch        714 non-null    int64   
 6   fare         714 non-null    float64 
 7   class        714 non-null    category
 8   who          714 non-null    object  
 9   adult_male   714 non-null    bool    
 10  embark_town  712 non-null    object  
 11  alive        714 non-null    object  
 12  alone        714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [63]:
most_freq = df['embark_town'].value_counts(dropna=True).idxmax()
most_freq

'Southampton'

In [64]:
df['embark_town'].fillna(most_freq, inplace=True)

In [65]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,Third,man,True,Southampton,no,True


In [66]:
rdf = df.copy()

In [68]:
# 카테고리형 데이터를 수치로 변환해주기 (더미사용하면 열이 늘어서 정확도 떨어지고 연산속도 저하)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(rdf['sex'])

LabelEncoder()

In [69]:
for col_name in rdf.columns[1:]:
    le.fit(rdf[col_name])
    rdf[col_name]=le.transform(rdf[col_name])

In [70]:
rdf['age'] = df['age']
rdf['fare'] = df['fare']

In [71]:
rdf.head()

,survived,pclass,sex,age,sibsp,parch,fare,class,who,adult_male,embark_town,alive,alone
0,0,2,1,22.0,1,0,7.2500,2,1,1,2,0,0
1,1,0,0,38.0,1,0,71.2833,0,2,0,0,1,0
2,1,2,0,26.0,0,0,7.9250,2,2,0,2,1,1
3,1,0,0,35.0,1,0,53.1000,0,2,0,2,1,0
4,0,2,1,35.0,0,0,8.0500,2,1,1,2,0,1


In [72]:
X = rdf[rdf.columns[1:12]]
y = rdf['survived']

In [73]:
# 설명 변수 데이터 정규화
X = preprocessing.StandardScaler().fit(X).transform(X)

In [74]:
# train data와 test data로 나누기
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=10)
print(X_train.shape)
print(X_test.shape)

(499, 11)
(215, 11)


In [75]:
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data 가지고 모형 학습
tree_model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [76]:
# test data를 가지고 y_hat을 예측 (분류)
y_hat = tree_model.predict(X_test)

In [77]:
df_y = pd.DataFrame({'y_hat':y_hat,'y_test':y_test})
df_y['차이'] = df_y['y_hat'] ==df_y['y_test']
df_y.head()

,y_hat,y_test,차이
728,0,0,True
555,0,0,True
426,1,1,True
278,0,0,True
617,0,0,True


In [78]:
len(df_y) - sum(df_y['차이'])

0

In [79]:
# 모형 성능 평가 - Confusion Matrix 계산
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)

[[125   0]
 [  0  90]]


In [80]:
# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       125
           1       1.00      1.00      1.00        90

    accuracy                           1.00       215
   macro avg       1.00      1.00      1.00       215
weighted avg       1.00      1.00      1.00       215

